In [0]:
# --- 1. Definição de Caminhos ---
nome_arquivo = "estados.csv"
caminho_final_arquivo = f"/Volumes/transacoes_db/copper/files/{nome_arquivo}"
caminho_temporario_spark = f"/Volumes/transacoes_db/copper/files/temp_chaves_pix_save"

# --- 2. Sua Query ---
query = """
select 
codigo_uf as id, nome, id_regiao, uf as sigla
from transacoes_db.copper.estados
"""

resultado_df = spark.sql(query)
count_query = resultado_df.count()
print(f"Quantidade de linhas na query: {count_query}")

# --- 3. Salvar em Arquivo Único (com .coalesce) ---
print(f"Salvando dados em diretório temporário: {caminho_temporario_spark}...")
resultado_df.coalesce(1).write \
    .format("csv") \
    .mode("overwrite") \
    .option("header", "true") \
    .save(caminho_temporario_spark)
print("Dados salvos temporariamente.")

# --- 4. Mover e Renomear o Arquivo (Com Correção) ---
try:
    # 1. Listar os arquivos no temp
    arquivos_no_temp = dbutils.fs.ls(caminho_temporario_spark)

    # 2. Encontrar o 'part-'
    arquivo_part = [f.path for f in arquivos_no_temp if f.name.startswith("part-") and f.name.endswith(".csv")]

    if not arquivo_part:
        print("Erro: Nenhum arquivo 'part-' foi encontrado no diretório temporário.")
    else:
        caminho_arquivo_origem = arquivo_part[0] 
        
        # --- !! CORREÇÃO IMPORTANTE !! ---
        # Garante que o destino não exista antes de mover.
        # Isso impede que o 'mv' mova o arquivo PARA DENTRO de um diretório existente.
        print(f"Limpando destino antigo (se existir): {caminho_final_arquivo}")
        dbutils.fs.rm(caminho_final_arquivo, recurse=True) # Deleta o arquivo OU diretório
        
        # 3. Mover e renomear o arquivo
        print(f"Movendo {caminho_arquivo_origem} para {caminho_final_arquivo}...")
        # recurse=False é o correto para mover um *arquivo*
        dbutils.fs.mv(caminho_arquivo_origem, caminho_final_arquivo, recurse=False)

        # 4. Limpar o diretório temporário
        print(f"Limpando diretório temporário: {caminho_temporario_spark}")
        dbutils.fs.rm(caminho_temporario_spark, recurse=True)

        print(f"\nResultado salvo com sucesso no ARQUIVO: {caminho_final_arquivo}")

        # --- 5. Validação ---
        print(f"\nLendo arquivo CSV final de: {caminho_final_arquivo}")
        csv_df = spark.read \
            .format("csv") \
            .option("header", "true") \
            .load(caminho_final_arquivo)

        count_csv = csv_df.count()
        print(f"Quantidade de linhas no arquivo CSV final: {count_csv}")
        
        if count_query == count_csv:
            print("Sucesso! A contagem de linhas bate com a query.")
        else:
            print("Atenção! A contagem de linhas está diferente.")

except Exception as e:
    print(f"Ocorreu um erro ao mover/renomear o arquivo: {e}")
    print("Verifique se o dbutils está disponível neste ambiente.")

In [0]:
%sql
select 
codigo_uf as id, nome, id_regiao, uf as sigla
from transacoes_db.copper.estados